## Loading the data

In [1]:
import os
import numpy as np
import json
import pandas as pd
import itertools
from tqdm import tqdm
import nltk
from nltk.tokenize import word_tokenize
import re
import matplotlib.pyplot as plt

In [2]:
def create_dataframe(dataset_path):
    """Function used to create a Pandas DataFrame containing specifications page titles

    Reads products specifications from the file system ("dataset_path" variable in the main function) and creates a Pandas DataFrame where each row is a
    specification. The columns are 'source' (e.g. www.sourceA.com), 'spec_number' (e.g. 1) and the 'page title'. Note that this script will consider only
    the page title attribute for simplicity.

    Args:
        dataset_path (str): The path to the dataset

    Returns:
        df (pd.DataFrame): The Pandas DataFrame containing specifications and page titles
    """

    print('>>> Creating dataframe...\n')
    columns_df = ['source', 'spec_number', 'spec_id', 'page_title']

    progressive_id = 0
    progressive_id2row_df = {}
    for source in tqdm(os.listdir(dataset_path)):
        for specification in os.listdir(os.path.join(dataset_path, source)):
            specification_number = specification.replace('.json', '')
            specification_id = '{}//{}'.format(source, specification_number)
            with open(os.path.join(dataset_path, source, specification)) as specification_file:
                specification_data = json.load(specification_file)
                page_title = specification_data.get('<page title>').lower()
                row = (source, specification_number, specification_id, page_title)
                progressive_id2row_df.update({progressive_id: row})
                progressive_id += 1
    df = pd.DataFrame.from_dict(progressive_id2row_df, orient='index', columns=columns_df)
    print('>>> Dataframe created successfully!\n')
    return df

In [3]:
df = create_dataframe('../datasets/unlabeled/2013_camera_specs')

  8%|▊         | 2/24 [00:00<00:01, 11.39it/s]

>>> Creating dataframe...



100%|██████████| 24/24 [00:11<00:00,  2.17it/s]

>>> Dataframe created successfully!



## Cleaning

In [4]:
df = df.drop(columns = ["source", "spec_number"], axis = 1)

In [5]:
from nltk.corpus import stopwords

stopWords = set(stopwords.words('english'))

In [6]:
stopWords = set(['itself', 'down', 'by', 'with', 'doesn', 'wouldn', 'other', 'ours', 'of', 'then', 'where', 'don', 'these', 'nor', 'she', "should've", 'won', 'ma', 'from', 'had', "you're", 'our', 'did', 'them', 'too', 'her', 'that', 'haven', 'after', "you'll", 'hers', 'because', 'yourself', 'against', 'mightn', 'as', 'll', 'whom', 'how', 'couldn', 'further', 'aren', "you'd", 'and', 'needn', "couldn't", 'those', 'to', "doesn't", "weren't", 'both', 'ourselves', 'in', 'which', 'yours', 'under', 'some', 'what', 'during', 'before', "needn't", "shan't", 'here', 'having', 'hasn', 'your', "hasn't", 'between', 'me', "she's", 'into', 'all', 'at', 'shan', 'who', 'o', 'an', 'very', 'can', 'you', 'shouldn', 'such', 'but', 'do', 'out', 'am', "shouldn't", 'above', 'wasn', 'or', 'were', 'own', 'didn', "you've", 'on', 'will', 'my', 'it', 'have', 'once', 'only', 'been', 'themselves', 'his', 'be', "mightn't", 'they', 'not', 'so', 'up', 'any', 'most', 'has', 'myself', 't', 'yourselves', 'isn', "it's", 'y', 'm', 'now', 'until', 're', 'there', 'their', 'mustn', "mustn't", 'again', 'being', 'hadn', 'doing', 'just', 'no', 'if', 've', "wasn't", "won't", 'we', 'below', 'does', 'more', 'this', 'should', "isn't", 'ain', "don't", 'i', "haven't", 'than', "didn't", 'are', 'about', 'off', 'him', 'for', 'few', "wouldn't", 'was', 'weren', 'why', 'he', "that'll", 'd', 'the', 'its', 'a', 'each', 'is', 'while', "aren't", 'when', 'theirs', 'same', 's', 'himself', 'herself', "hadn't", 'through', 'over'])

In [7]:
punctuation = "!#$%&'()*+,-./:;<=>?@[\]^_`{|}~€£¥₹₽"

In [8]:
def replace_punctuation(word):
    return ''.join(c for c in word if c not in punctuation)

In [9]:
df["page_title"] = df["page_title"].apply(lambda x : [i.lower() for i in list(map(lambda y: replace_punctuation(y), word_tokenize(x))) if i and i.lower() not in stopWords])

## Model Words

In [10]:
pattern = re.compile("(\S*[A-Za-z]\S*[0-9]\S*|\S*[0-9]\S*[A-Za-z]\S*)")

In [11]:
## In the data replace lumix with panasonic

In [12]:
brands = ['360fly', 'acer', 'achiever', 'acorn', 'actionpro', 'activeon', 'aee', 'agfa', 'agfaphoto', 'aiptek', 'akaso', 'alpine', 'alpine', 'amkov', 'andoer', 'annke', 'ansco', 'apeman', 'apex', 'apple', 'archos', 'argus', 'arlo', 'arri', 'axis', 'bell', 'benq', 'blackmagic', 'blackmagic', 'bosch', 'bower', 'brinno', 'brookstone', 'browning', 'cambo', 'campark', 'canon', 'carl', 'casio', 'celestron', 'chinon', 'cisco', 'cobra', 'coleman', 'concord', 'contax', 'contour', 'covert', 'craig', 'crayola', 'creative', 'creative', 'crosstour', 'crumpler', 'datavideo', 'delkin', 'dell', 'digitrex', 'discovery', 'disney', 'dji', 'd-link', 'domke', 'dörr', 'dragon', 'dxg', 'dxo', 'easypix', 'elecom', 'elmo', 'emerson', 'energizer', 'epson', 'fisher-price', 'flip', 'flir', 'foscam', 'fotoman', 'fotopro', 'fuji', 'fujifilm', 'fujinon', 'garmin', 'gateway', 'godox', 'goodmans', 'google', 'gopro', 'grundig', 'hahnel', 'hamilton', 'hasselblad', 'hello', 'herofiber', 'hitachi', 'holga', 'horseman', 'hoya', 'htc', 'huawei', 'ikelite', 'ilford', 'impossible', 'innovage', 'insignia', 'insta360', 'intel', 'intova', 'ion', 'iris', 'jazz', 'jenoptik', 'jjrc', 'jvc', 'kaiser', 'kenko', 'keyence', 'king', 'kitvision', 'kodak', 'konica', 'kyocera', 'leaf', 'lego', 'leica', 'lenovo', 'lexibook', 'linhof', 'liquid', 'little', 'logitech', 'lomography', 'lowepro', 'ltl', 'lytro', 'maginon', 'magnavox', 'mamiya', 'manfrotto', 'marshall', 'marumi', 'mattel', 'maxell', 'meade', 'medion', 'memorex', 'mercury', 'metz', 'microsoft', 'microtek', 'midland', 'minolta', 'minox', 'monster', 'motorola', 'moultrie', 'mustek', 'nabi', 'neewer', 'nest', 'netgear', 'night', 'nikkon', 'nikkor', 'nikon', 'nilox', 'nintendo', 'nippon', 'nokia', 'norcent', 'olympus', 'optech', 'ordro', 'oregon', 'packard', 'palm', 'panasonic', 'parrot', 'pelco', 'pentacon', 'pentax', 'phase', 'philips', 'philips', 'phoenix', 'pioneer', 'playskool', 'polaroid', 'polarpro', 'praktica', 'premier', 'promaster', 'proscan', 'pyle', 'radioshack', 'raymarine', 'raynox', 'rca', 'ricoh', 'ring', 'rode', 'rokinon', 'rollei', 'ryobi', 'sakar', 'samsung', 'sandisk', 'sanyo', 'schneider', 'schneider', 'schneider', 'scosche', 'seasea', 'sealife', 'sharp', 'sharper', 'sigma', 'sinar', 'sipix', 'sjcam', 'sony', 'soocoo', 'stealth', 'superheadz', 'svp', 'swann', 'tamrac', 'tamron', 'technika', 'tenba', 'think', 'thule', 'tokina', 'tomy', 'toshiba', 'transcend', 'traveler', 'trust', 'verbatim', 'vibe', 'victure', 'vistaquest', 'vivitar', 'voigtländer', 'vtech', 'vupoint', 'walimex', 'wyze', 'xiaomi', 'xit', 'xtreme', 'yashica', 'zeiss']

In [13]:
df["page_title"] = df["page_title"].apply(lambda line : list(set(filter(lambda word : bool(pattern.match(word)) or word in brands,line))))

In [14]:
df.head()

,spec_id,page_title
0,www.wexphotographic.com//154,"[aw120, nikon, vna593e1]"
1,www.wexphotographic.com//553,"[9148b007aa, canon]"
2,www.wexphotographic.com//601,"[p10nc12730a, fuji, s1]"
3,www.wexphotographic.com//197,"[s5300, nikon, vna540e1]"
4,www.wexphotographic.com//178,"[s8600, fuji, p10nc12690a]"


In [15]:
df["brand"] = ""

In [16]:
# See how many products have more than one brand
for index, row in df.iterrows():
    for brand in row["page_title"]:
        if brand in brands:
            df["brand"].iloc[index] = brand

In [17]:
df.head()

,spec_id,page_title,brand
0,www.wexphotographic.com//154,"[aw120, nikon, vna593e1]",nikon
1,www.wexphotographic.com//553,"[9148b007aa, canon]",canon
2,www.wexphotographic.com//601,"[p10nc12730a, fuji, s1]",fuji
3,www.wexphotographic.com//197,"[s5300, nikon, vna540e1]",nikon
4,www.wexphotographic.com//178,"[s8600, fuji, p10nc12690a]",fuji


In [18]:
for index, row in df.iterrows():
    if row["brand"] == "":
        df["brand"].iloc[index] = "unbranded"

In [19]:
# TODO manage entries with multiple brands

In [20]:
df = df.groupby(["brand"])

## Algorith for every brand

In [ ]:
import time
t = time.time()

for group_name, group in df:
    print("CALCULATING FOR BRAND = ", group_name)
    merged = group.drop(columns=["brand"], axis = 1)
    merged = (merged.merge(merged, on=merged.assign(key_col=1)['key_col'], suffixes=('', '_right'))
 .query('spec_id != spec_id_right') # filter out joins on the same row
 .reset_index(drop=True))
    merged.drop(columns = ["key_0"], axis = 1, inplace=True)
    merged["zipped"] = tuple(map(lambda line : sorted(line), list(zip(merged["spec_id"], merged["spec_id_right"]))))
    merged.drop_duplicates("zipped", inplace=True)
    merged.drop(columns=["zipped"], inplace=True)
    merged.rename(columns = {"spec_id" : "left_spec_id", "spec_id_right" : "right_spec_id"}, inplace=True)
    merged.reset_index(inplace=True)
    #
    labels = []
    print("NUMBER OF COMPARISONS: ", len(merged))
    commons = 0
    for index, row in merged.iterrows():
        if index % 100000 == 0 and index != 0:
            print(index)
        for spec in row["page_title"]:
            if spec in row["page_title_right"]:
                commons += 1
        if commons >= 2:
            labels.append(1)
        else:
            labels.append(0)
        commons = 0
    merged["label"] = labels
    del labels
    merged = merged.loc[merged['label'] == 1]
    merged.drop(columns=["page_title", "page_title_right", "label"], axis = 1).to_csv("../datasets/created/brand_{}_compact_matches_labeled.csv".format(group_name),index = False)

elapsed = time.time() - t
print("ALL TOGETHER IT TOOK : ", elapsed, "SECONDS")

CALCULATING FOR BRAND =  aee
NUMBER OF COMPARISONS:  1
CALCULATING FOR BRAND =  agfa
NUMBER OF COMPARISONS:  1
CALCULATING FOR BRAND =  agfaphoto
NUMBER OF COMPARISONS:  0
CALCULATING FOR BRAND =  aiptek
NUMBER OF COMPARISONS:  55
CALCULATING FOR BRAND =  alpine
NUMBER OF COMPARISONS:  1
CALCULATING FOR BRAND =  apex
NUMBER OF COMPARISONS:  1
CALCULATING FOR BRAND =  apple
NUMBER OF COMPARISONS:  28
CALCULATING FOR BRAND =  argus
NUMBER OF COMPARISONS:  28
CALCULATING FOR BRAND =  axis
NUMBER OF COMPARISONS:  6
CALCULATING FOR BRAND =  bell
NUMBER OF COMPARISONS:  300
CALCULATING FOR BRAND =  benq
NUMBER OF COMPARISONS:  105
CALCULATING FOR BRAND =  blackmagic
NUMBER OF COMPARISONS:  28
CALCULATING FOR BRAND =  bosch
NUMBER OF COMPARISONS:  15
CALCULATING FOR BRAND =  brinno
NUMBER OF COMPARISONS:  3
CALCULATING FOR BRAND =  canon
